## **Resultado final - "Emergencias UNGRD"**
----
Este notebook integra los dos CSVs previos en uno solo. Se realizó un profundo proceso de limpieza en, aproximadamente, 18 fuentes de datos (Excel y JSON) que derivo en el CSV final que resulta del procesamiento de datos realizado en este notebook.

**Elaborado con base en**
* [​​​​​​​​​​​​Consolidado anual de emergencias ​​| UNGRD](https://portal.gestiondelriesgo.gov.co/Paginas/Consolidado-Atencion-de-Emergencias.aspx)
* [Emergencias UNGRD | Datos Abiertos Colombia](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Emergencias-UNGRD-/wwkg-r6te/about_data)

#### ***Importación de paquetes***

In [34]:
import pandas as pd

#### ***Cargar los CSVs mediante Pandas***

In [35]:
emergenciasConsolidados = pd.read_csv('../CSVs/EmergenciasCC(2003-2018, 2023)_RegionPacifico.csv')
emergenciasDatosAbiertos = pd.read_csv('../CSVs/EmergenciasCC(2019-2022)_RegionPacifico.csv')

#### ***Imprimir una muestra de los datos obtenidos***

##### **Obtenido desde los consolidados *(sin filtrar aún por evento)***

In [36]:
emergenciasConsolidados.sample(5)

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
1242,2004.0,11.0,2004-11-15,INUNDACION,DEPARTAMENTO,MAGDALENA,0,0,0,0,0,0,0,0.0
14753,2012.0,6.0,2012-06-05,INCENDIO FORESTAL,BELLO,ANTIOQUIA,0,0,0,0,0,0,0,8.0
5630,2008.0,11.0,2008-11-16,INUNDACION,PUERTO SALGAR,CUNDINAMARCA,0,0,0,198,47,0,0,0.0
29689,2016.0,3.0,2016-03-26,INCENDIO FORESTAL,CAQUEZA,CUNDINAMARCA,0,0,0,0,0,0,0,6.0
33266,2017.0,4.0,2017-04-19,DESLIZAMIENTO,BUENOS AIRES,CAUCA,0,0,0,5,1,0,1,0.0


##### **Obtenido desde el dataset de Datos Abiertos**

In [37]:
emergenciasDatosAbiertos.sample(5)

,AÑO,MES,FECHA,DEPARTAMENTO,MUNICIPIO,EVENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
2407,2022,6,2022-06-17,VALLE DEL CAUCA,YUMBO,MOVIMIENTO EN MASA,0,0,0,0,0,0,2,0.0
3033,2022,4,2022-04-13,CAUCA,TORIBIO,MOVIMIENTO EN MASA,0,0,0,137,38,0,3,0.0
826,2020,11,2020-11-20,CAUCA,SUAREZ,MOVIMIENTO EN MASA,0,0,0,27,5,0,0,0.0
373,2019,8,2019-08-28,CAUCA,LA SIERRA,INCENDIO DE COBERTURA VEGETAL,0,0,0,0,0,0,0,0.0
2585,2022,9,2022-09-10,NARIÑO,CONSACA,INCENDIO DE COBERTURA VEGETAL,0,0,0,0,0,0,0,2.0


#### ***Concatenar los dos dataframes***

In [38]:
# Lista de nombres de los dataframes
dataframes = [emergenciasConsolidados, emergenciasDatosAbiertos]

# Concatenar los dataframes en uno solo
emergencias = pd.concat(dataframes)

emergencias.sample(5)

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
2365,2022.0,5.0,2022-05-18,INUNDACION,LINARES,NARIÑO,0,0,0,724,181,12,20,0.0
11816,2011.0,10.0,2011-10-03 00:00:00,INCENDIO ESTRUCTURAL,QUIBDO,CHOCO,0,0,0,10,2,2,0,0.0
40335,2023.0,2.0,2023-02-17,INCENDIO FORESTAL,RIOHACHA,LA GUAJIRA,0,0,0,0,0,0,0,2.0
39606,2023.0,1.0,2023-01-30,INCENDIO FORESTAL,VENADILLO,TOLIMA,0,0,0,0,0,0,0,4.0
36653,2018.0,4.0,2018-04-15,COLAPSO,FLORENCIA,CAQUETA,0,8,0,8,0,0,0,0.0


## **Limpieza de datos**
---

### ***Consolidados - UNGRD***

#### ***Escogemos las filas con los eventos de nuestro interés***
En especial aquellos que son impactados fuertemente por el cambio climático.

In [39]:
emergencias["EVENTO"] = emergencias["EVENTO"].fillna("")

emergencias["EVENTO"] = emergencias["EVENTO"].str.upper()

emergencias["EVENTO"] = emergencias["EVENTO"].str.strip()

# Diccionario de reemplazo
diccionario_reemplazos = {
    "INUNDACION": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "INUNDACIÓN": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "AVENIDA TORRENCIAL": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE SUBITA": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE SÚBITA": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "CRECIENTE": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "LLUVIAS": "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "VENDAVAL-INCENDIO ESTRUCTURAL": "VENDAVAL",
    "DELIZAMIENTO": "DESLIZAMIENTO",
    "MOVIMIENTO EN MASA": "DESLIZAMIENTO",
    "REMOCIÓN EN MASA": "DESLIZAMIENTO",
    "REMOCION EN MASA": "DESLIZAMIENTO",
    "MAREJADA": "MAREJADAS/MAR DE LEVA",
    "MAREJADAS": "MAREJADAS/MAR DE LEVA",
    "MAR DE LEVA": "MAREJADAS/MAR DE LEVA",
    "MAR DELEVA": "MAREJADAS/MAR DE LEVA",
    "EROSION-INCENDIO ESTRUCTURAL": "EROSION",
    "EROSION COSTERA": "EROSION",
    "FENOMENO ATMOSFERICO": "TEMPORAL",
    "CICLON TROPICAL: DEPRESION/TORMENTA/HURACAN": "DEPRESION/TORMENTA/HURACAN",
    "CICLÓN, TROPICAL, DEPRESIÓN/TORMENTA/HURACÁN": "DEPRESION/TORMENTA/HURACAN",
    "TORMENTA ELECTRICA": "DEPRESION/TORMENTA/HURACAN",
    "TORNADO": "DEPRESION/TORMENTA/HURACAN",
    "DEPRESION TROPICAL": "DEPRESION/TORMENTA/HURACAN",
    "HURACAN": "DEPRESION/TORMENTA/HURACAN",
}

# Aplicar los cambios
emergencias["EVENTO"] = emergencias["EVENTO"].replace(diccionario_reemplazos)

In [40]:
# Lista de eventos a filtrar (ajustada a los nombres proporcionados)
eventos_a_filtrar = [
    "AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA",
    "INCENDIO FORESTAL",
    "DESLIZAMIENTO",
    "VENDAVAL",
    "SEQUIA",
    "DEPRESION/TORMENTA/HURACAN",
    "GRANIZADA",
    "TEMPORAL",
    "EROSION",
    "HELADA",
    "MAREJADAS/MAR DE LEVA"
]

# Filtrar el DataFrame
emergencias = emergencias[emergencias["EVENTO"].isin(eventos_a_filtrar)]

# Mostrar el DataFrame filtrado
emergencias

,AÑO,MES,FECHA,EVENTO,MUNICIPIO,DEPARTAMENTO,FALLECIDOS,HERIDOS,DESAPARECIDOS,PERSONAS,FAMILIAS,VIVIENDAS,VIAS,HECTAREAS
0,2003.0,1.0,2003-01-01,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,BAGADO,CHOCO,1,0,0,700,175,0,0,0.0
1,2003.0,1.0,2003-01-01,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,MEDIO ATRATO,CHOCO,0,0,0,3161,691,0,0,0.0
2,2003.0,1.0,2003-01-01,VENDAVAL,LITORAL DEL SAN JUAN,CHOCO,0,0,0,995,199,0,0,0.0
3,2003.0,1.0,2003-01-03,INCENDIO FORESTAL,LA TEBAIDA,QUINDIO,0,0,0,0,0,0,0,0.0
4,2003.0,1.0,2003-01-03,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,LLORO,CHOCO,0,0,0,1076,269,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3544,2022.0,12.0,2022-12-26,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,ROSAS,CAUCA,0,0,0,0,0,0,0,0.0
3545,2022.0,12.0,2022-12-26,DESLIZAMIENTO,ROSAS,CAUCA,0,0,0,7,2,0,0,0.0
3546,2022.0,12.0,2022-12-26,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,PADILLA,CAUCA,0,0,0,0,0,0,0,0.0
3572,2022.0,11.0,2022-11-25,AVENIDA TORRENCIAL/INUNDACION/CRECIENTE SUBITA,DEPARTAMENTO,VALLE DEL CAUCA,0,0,0,0,0,0,0,0.0


#### ***Arregar formato en la fecha***

In [41]:
emergencias = emergencias.dropna()
emergencias = emergencias[emergencias != '']

if emergencias is not None:
    emergencias['AÑO'] = emergencias['AÑO'].astype(int, errors='ignore')
    emergencias['MES'] = emergencias['MES'].astype(int, errors='ignore')

# Eliminar la hora de la columna de fecha
emergencias['FECHA'] = pd.to_datetime(emergencias['FECHA'], format='ISO8601')
emergencias['FECHA'] = emergencias['FECHA'].dt.date

# Organizar los años y los meses de manera ascendente
emergencias = emergencias.sort_values(by=['AÑO', 'MES'], ascending=[True, True])

#### ***Nos enfocaremos en los departamentos de la región Pacífico***

In [42]:
emergencias["DEPARTAMENTO"].value_counts()

DEPARTAMENTO
CUNDINAMARCA               5379
CAUCA                      3190
TOLIMA                     2750
NARIÑO                     2636
SANTANDER                  2532
ANTIOQUIA                  2498
HUILA                      2300
BOYACA                     2168
VALLE DEL CAUCA            1677
RISARALDA                  1465
CHOCO                      1453
CASANARE                   1201
NORTE DE SANTANDER         1097
MAGDALENA                  1029
BOLIVAR                     988
CESAR                       949
CORDOBA                     902
ATLANTICO                   891
META                        855
QUINDIO                     796
CALDAS                      775
SUCRE                       622
CAQUETA                     443
LA GUAJIRA                  426
PUTUMAYO                    409
ARAUCA                      326
BOGOTA, D.C.                265
VALLE                       199
VICHADA                     173
GUAVIARE                    137
GUAJIRA                    

In [43]:
diccionario_reemplazos = {
    "VALLE ": "VALLE DEL CAUCA",
    "VALLE": "VALLE DEL CAUCA"
}

emergencias["DEPARTAMENTO"] = emergencias["DEPARTAMENTO"].replace(diccionario_reemplazos)

departamentos_deseados = ["NARIÑO", "CAUCA", "VALLE DEL CAUCA", "CHOCO"]

emergencias = emergencias[emergencias["DEPARTAMENTO"].isin(departamentos_deseados)]

### ***Guardar como CSV***
---

In [44]:
emergencias.to_csv('../EmergenciasCC(2003-2023)_RegionPacifico.csv', index=False)